In [1]:
#Import Python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import statsmodels.api as sm
import array

In [2]:
#Download csv files
total_sap_flow_LDP1 = pd.read_csv('11_10_23_DB_LDP1_Total_Sap_Flow.csv', parse_dates=['Date/Time'])
total_sap_flow_LDP = pd.read_csv('11_10_23_DB_LDP_Total_Sap_Flow.csv', parse_dates=['Date/Time'])
May_Oct_Met = pd.read_csv('11_10_23_LDP_MET.csv', parse_dates=['DateTime'])
Precip_Met = pd.read_csv('LDP_Met_FieldSeason_Precip.csv', parse_dates=['DateTime'])

In [3]:
total_sap_flow_LDP1_filter = total_sap_flow_LDP1.drop(total_sap_flow_LDP1[total_sap_flow_LDP1['Date/Time'] == '1/0/1900 0:00'].index)
total_sap_flow_LDP_filter = total_sap_flow_LDP.drop(total_sap_flow_LDP[total_sap_flow_LDP['Date/Time'] == '1/0/1900 0:00'].index)

In [4]:
total_sap_flow_LDP1_filter['Date/Time'] = pd.to_datetime(total_sap_flow_LDP1_filter['Date/Time'])
total_sap_flow_LDP_filter['Date/Time'] = pd.to_datetime(total_sap_flow_LDP_filter['Date/Time'])
May_Oct_Met['DateTime'] = pd.to_datetime(May_Oct_Met['DateTime'])
Precip_Met['DateTime'] = pd.to_datetime(Precip_Met['DateTime'])

total_sap_flow_LDP1_filter.set_index('Date/Time', inplace=True)
total_sap_flow_LDP_filter.set_index('Date/Time', inplace=True)
May_Oct_Met.set_index('DateTime', inplace=True)
Precip_Met.set_index('DateTime', inplace=True)

In [5]:
May_Oct_met_station = May_Oct_Met['2023-05-25 18:00:00':'2023-10-31 23:00:00']
Precip_met = Precip_Met['2023-05-25 18:00:00':'2023-10-31 23:00:00']
May_Oct_LDP = (total_sap_flow_LDP_filter['2023-05-25 18:00:00':'2023-10-31 23:00:00'])
May_Oct_LDP1 = (total_sap_flow_LDP1_filter['2023-05-25 18:00:00':'2023-10-31 23:00:00'])

In [6]:
#AE, VWC, AT, & RH daily averages
Net_Radiation_Shortwave1 = (May_Oct_met_station['Solar_Radiation_Shortwave_Up_Average_W/m2'])
Net_Radiation_Shortwave2 = (May_Oct_met_station['Solar_Radiation_Shortwave_Down_Average_W/m2'])
Net_Radiation_Shortwave = (Net_Radiation_Shortwave1+Net_Radiation_Shortwave2)
Net_Radiation_Longwave = (May_Oct_met_station['Solar_Radiation_Longwave_Up_Average_W/m2']+May_Oct_met_station['Solar_Radiation_Longwave_Down_Average_W/m2'])
Available_Energy = (Net_Radiation_Shortwave-Net_Radiation_Longwave)
#AE_diurnal = (Available_Energy.groupby(Available_Energy.index.day))

Volumetric_Water_Content_West_LDP = May_Oct_met_station['Volumetric_Water_Content1_Soil_Moisture_8cm_west_cm3/cm3']
Volumetric_Water_Content_East_LDP = May_Oct_met_station['Volumetric_Water_Content2_Soil_Moisture_8cm_east_cm3/cm3']

Air_T = (May_Oct_met_station['Air_Temp_Average_DegC'])
R_H = (May_Oct_met_station['Relative_Humidity_Average_%'])

In [7]:
#Calculating Vapor Pressure Defecit (VPD)

#First saturated vapor pressure
sat_vp = 0.6108 * np.exp(17.27 * Air_T / (Air_T + 237.3))


#Now actual vapor pressure 
actual_vp = R_H / 100 * sat_vp


#VPD
VPD = sat_vp-actual_vp
#VPD_diurnal = (VPD.groupby(VPD.index.day))

In [8]:
#Add to the dataframe
May_Oct_met_station['VPD'] = VPD
May_Oct_met_station['Available_Energy'] = Available_Energy
May_Oct_met_station['Volumetric_Water_Content_West_LDP'] = Volumetric_Water_Content_West_LDP
May_Oct_met_station['Volumetric_Water_Content_East_LDP'] = Volumetric_Water_Content_East_LDP
May_Oct_met_station['Air_T'] = Air_T
May_Oct_met_station['R_H'] = R_H

C:\Users\Delaney\AppData\Local\Temp\ipykernel_10592\3034716751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  May_Oct_met_station['VPD'] = VPD
C:\Users\Delaney\AppData\Local\Temp\ipykernel_10592\3034716751.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  May_Oct_met_station['Available_Energy'] = Available_Energy
C:\Users\Delaney\AppData\Local\Temp\ipykernel_10592\3034716751.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [9]:
May_Oct_met_station

,Solar_Radiation_Shortwave_Up_Average_W/m2,Solar_Radiation_Shortwave_Down_Average_W/m2,Solar_Radiation_Longwave_Up_Average_W/m2,Solar_Radiation_Longwave_Down_Average_W/m2,Net_Radiation_Incoming_Outgoing_Average_DegC,Net_Radiation_Temperature_Average_Kelvin,Net_Radiation_Longwave_Co_Average_W/m2,Net_Radiation_Longwave_Co_Average_W/m2.1,Net_Radiation_Shortwave_Average_W/m2,Soil_Moisture1_period_8cm_west,...,Hz_Wind_Speed_m/s,Wind_Speed_Mean_Velocity_Magnitude_m/s,Precip_Ridgetop_mm,Precip_Hillslope_mm,VPD,Available_Energy,Volumetric_Water_Content_West_LDP,Volumetric_Water_Content_East_LDP,Air_T,R_H
DateTime,,,,,,,,,,,,,,,,,,,,,
2023-05-25 18:00:00,454.700,67.680,-64.94,17.550,17.410,290.6,339.2,421.7,454.600,21.31,...,0,0,6.478,1.981,0.883604,569.770,0.117,0.132,15.480,49.76
2023-05-25 19:00:00,277.700,44.150,-67.83,8.890,15.970,289.1,328.3,405.1,277.700,21.31,...,0,0,6.479,1.981,0.829238,380.790,0.117,0.132,14.780,50.68
2023-05-25 20:00:00,117.400,16.000,-68.60,-2.724,15.040,288.2,322.5,388.4,117.500,21.29,...,0,0,6.480,1.981,0.772582,204.724,0.117,0.131,14.270,52.51
2023-05-25 21:00:00,1.446,1.183,-64.79,-6.314,11.980,285.1,310.0,368.5,3.532,21.28,...,0,0,6.391,1.981,0.651438,73.733,0.117,0.131,12.430,54.85
2023-05-25 22:00:00,-3.882,0.612,-64.61,-9.130,11.120,284.3,305.6,361.1,0.000,21.28,...,0,0,6.483,1.981,0.656054,70.470,0.117,0.130,12.070,53.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-31 19:00:00,-3.107,0.734,-58.58,-7.868,4.393,277.5,277.9,328.6,0.000,21.02,...,0,0,172.000,2.155,0.528692,64.075,0.111,0.135,5.255,40.46
2023-10-31 20:00:00,-3.852,0.442,-65.36,-9.270,4.227,277.4,270.3,326.4,0.000,21.01,...,0,0,172.000,2.155,0.501930,71.220,0.110,0.135,5.015,42.52
2023-10-31 21:00:00,-4.799,0.607,-79.90,-10.140,3.197,276.3,250.8,320.5,0.000,21.00,...,0,0,172.000,2.155,0.449242,85.848,0.110,0.135,4.098,45.14


In [10]:
May_Oct_LDP

,Total Sap Flow (L/hr or kg/hr)
Date/Time,
2023-05-25 18:00:00,0.56
2023-05-25 19:00:00,0.54
2023-05-25 20:00:00,0.47
2023-05-25 21:00:00,0.36
2023-05-25 22:00:00,0.27
...,...
2023-10-31 19:00:00,0.20
2023-10-31 20:00:00,0.21
2023-10-31 21:00:00,0.16


In [11]:
array_VPD = np.asarray(May_Oct_met_station['VPD'], dtype=object)
array_RH = np.asarray(May_Oct_met_station['R_H'], dtype=object)
array_AT = np.asarray(May_Oct_met_station['Air_T'], dtype=object)
array_AE = np.asarray(May_Oct_met_station['Available_Energy'], dtype=object)
array_VWC_E = np.asarray(May_Oct_met_station['Volumetric_Water_Content_East_LDP'], dtype=object)
array_VWC_W = np.asarray(May_Oct_met_station['Volumetric_Water_Content_West_LDP'], dtype=object)

array_LDP = np.asarray(May_Oct_LDP, dtype=object)
array_LDP1 = np.asarray(May_Oct_LDP1, dtype=object)

display(array_LDP.reshape(-1,1))
#array_LDP1.reshape(-1,1)

#array_VPD.reshape(-1,1)
#array_RH.reshape(-1,1)
#array_AT.reshape(-1,1)
#array_AE.reshape(-1,1)
#array_VWC_E.reshape(-1,1)
#array_VWC_W.reshape(-1,1)

array([[0.56],
       [0.54],
       [0.47],
       ...,
       [0.16],
       [0.16],
       [0.16]], dtype=object)

In [12]:
#Multiple Linear Regression SF LDP1

#These are the independent variables
x = (array_VPD.reshape(-1,1),
array_RH.reshape(-1,1),
array_AT.reshape(-1,1),
array_AE.reshape(-1,1),
array_VWC_E.reshape(-1,1),
array_VWC_W.reshape(-1,1))

#This is the dependent variable
y = (array_LDP.reshape(-1,1), array_LDP1.reshape(-1,1))

#Modeling the linear regression
regr = linear_model.LinearRegression() 
regr.fit(x, y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

#Add constant
x = sm.add_constant(x) 

model = sm.OLS(y, x).fit()
predictions = model.predict(x)

#print the multiple linear regression output
MLR_model = model.summary()

import warnings
warnings.filterwarnings("ignore")

print(print_model)

C:\Users\Delaney\anaconda3\lib\site-packages\sklearn\utils\validation.py:964: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(


ValueError: Found array with dim 3. Estimator expected <= 2.